In [57]:
import dolfin as dolf
from math import pi, sqrt
import matplotlib.pyplot as plt
import numpy as np
from helmholtz_pkg.passive_flame import PassiveFlame
from helmholtz_pkg.flame_transfer_function import n_tau
from helmholtz_pkg.active_flame import ActiveFlame
from helmholtz_pkg.eigensolvers import fixed_point_iteration_pep
from helmholtz_pkg.eigenvectors import normalize_eigenvector

import params

In [58]:
def mshr(el):

    mesh = dolf.UnitIntervalMesh(el)

    def l_boundary_func(x, on_boundary):
        x = x[0]
        return on_boundary and dolf.near(x, 0.)

    def r_boundary_func(x, on_boundary):
        x = x[0]
        return on_boundary and dolf.near(x, 1.)

    boundaries = dolf.MeshFunction('size_t', mesh, mesh.topology().dim() - 1)

    l_boundary = dolf.AutoSubDomain(l_boundary_func)
    r_boundary = dolf.AutoSubDomain(r_boundary_func)

    l_boundary.mark(boundaries, 1)
    r_boundary.mark(boundaries, 2)

    # ________________________________________________________________________________

    def fl_subdomain_func(x):
        x = x[0]
        x_f = params.x_f[0][0]
        a_f = params.a_f
        return x_f - a_f - dolf.DOLFIN_EPS <= x <= x_f + a_f + dolf.DOLFIN_EPS

    subdomains = dolf.MeshFunction('size_t', mesh, mesh.topology().dim())

    subdomains.set_all(1)

    fl_subdomain = dolf.AutoSubDomain(fl_subdomain_func)
    fl_subdomain.mark(subdomains, 0)

    return mesh, boundaries, subdomains

In [59]:
degree = 1

mesh, boundaries, subdomains = mshr(30)

boundary_conditions = {1: {'Robin': params.Y_in},  # inlet
                       2: {'Robin': params.Y_out}}  # outlet

foo = PassiveFlame(mesh, boundaries, boundary_conditions,
                   c=params.c,
                   degree=degree)
foo.assemble_A()
foo.assemble_B()
foo.assemble_C()


ftf = n_tau(params.n, params.tau)
D = ActiveFlame(mesh, subdomains,
                    params.x_f, params.x_r, params.rho_in, 1., 1., ftf,
                    degree=degree)
D.assemble_submatrices() 
D_11, D_12, D_21, D_22 = D.submatrices
D_mat = D_11 + D_12 + D_21 + D_22
D_mat_  = D_mat.getValues(range(D_mat.size[0]), range(D_mat.size[1]))
np.flatnonzero(D_mat_)

array([2776, 2777, 2778, 2779, 2838, 2839, 2840, 2841, 2900, 2901, 2902,
       2903, 2962, 2963, 2964, 2965])

## MATRIX D

In [60]:
def assemble_left_vector(fl):

    (v_1, v_2) = dolf.TestFunction(foo.function_space)
    dx = dolf.Measure('dx', subdomain_data=subdomains)

    V = dolf.FunctionSpace(mesh, 'CG', 1)
    const = dolf.interpolate(dolf.Constant(1), V)
    V_fl = dolf.assemble(const * dx(fl))

    a_1 = dolf.assemble(v_1  / V_fl * dx(fl))
    a_2 = dolf.assemble(v_2  / V_fl * dx(fl))

#     def helper_func(v, dofmap):
#         indices = np.flatnonzero(v) # Returns the indices which are nonzero
#         values = v[indices]
#         my_list = []
#         for index, value in zip(indices, values):
#             my_list.append([dofmap.dofs()[index], value])
#         v = np.array(my_list)
#         return v
    
#     dofmap = foo.function_space.dofmap()
#     a_1 = helper_func(a_1, dofmap)
#     a_2 = helper_func(a_2, dofmap)
    
    a = (a_1, a_2)

    return a

a = assemble_left_vector(0)
a_vec = a[0] + a[1]


In [61]:
a_vec = a_vec[:]

In [62]:
def assemble_right_vector(x):

    v = np.array([[0, 0, 1]])  # row
    dimension = mesh.geometric_dimension()
    if dimension == 1:
        v = np.array([[1]])
    elif dimension == 2:
        v = np.array([[1, 0]])
    # else:  # elif dimension == 3:
    #     pass
    v = v.transpose()  # column

    b = [np.array([]), np.array([])]

    cell_index = mesh.bounding_box_tree().compute_first_entity_collision(dolf.Point(x))
    # print("cell index is: ",cell_index, "max cell number: ",mesh.num_entities(mesh.topology().dim()))
    if cell_index <= mesh.num_entities(mesh.topology().dim()):
        # print("Cell is in Mesh")
        cell = dolf.Cell(mesh, cell_index)

        b = []

        for j in range(2):

            dofmap = foo.function_space.sub(j).dofmap()
            cell_dofs = dofmap.cell_dofs(cell_index)

            element = foo.function_space.sub(j).element()
            d_dx = element.evaluate_basis_derivatives_all(1, x, cell.get_vertex_coordinates(), cell.orientation())

            d_dx = d_dx.reshape((len(cell_dofs), -1))

            d_dv = np.dot(d_dx, v)
            d_dv = d_dv[:, 0]

            my_list = []
            print("cell~_dofs: ",cell_dofs )
            for i, dof in enumerate(cell_dofs):
                my_list.append([dofmap.tabulate_local_to_global_dofs()[dof], d_dv[i]])
                print("tabulate_local_to_global_dofs()[dof] ",dofmap.tabulate_local_to_global_dofs()[dof] )
            my_vec = np.array(my_list)
            #print(my_vec)
            b.append(my_vec)
        #
        # else:
        #
        #     pass
    return (*b, )

b = assemble_right_vector(params.x_r[0][0])
b

cell~_dofs:  [50 48]
tabulate_local_to_global_dofs()[dof]  50
tabulate_local_to_global_dofs()[dof]  48
cell~_dofs:  [51 49]
tabulate_local_to_global_dofs()[dof]  51
tabulate_local_to_global_dofs()[dof]  49


(array([[ 50., -30.],
        [ 48.,  30.]]),
 array([[ 51., -30.],
        [ 49.,  30.]]))

In [63]:
# Append the derivative values to the matrix b
b_vec = np.zeros(2*(mesh.num_vertices()))
for i in range(0,len(b)):
    for dv in b[i]:
        b_vec[int(dv[0])] = dv[1]

In [64]:
print(len(a_vec), len(b_vec))
multiplication = np.outer(a_vec , b_vec)
print(multiplication.shape)
np.flatnonzero(multiplication)

62 62
(62, 62)


array([2776, 2777, 2778, 2779, 2838, 2839, 2840, 2841, 2900, 2901, 2902,
       2903, 2962, 2963, 2964, 2965])

In [65]:
np.set_printoptions(threshold=np.inf)
multiplication

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,  -0.,  -0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,  -0.,  -0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          

In [66]:
np.array_equal(multiplication,D_mat_)

True

In [67]:
### VERY INEFFICIENT
indices = np.nonzero(D_mat_)
D_values = np.flatnonzero(D_mat_)

from petsc4py import PETSc
multiplication.shape
D_ekrem = PETSc.Mat().create()
D_ekrem.setSizes(multiplication.shape)
D_ekrem.setType('aij')
D_ekrem.setUp()

# D_ekrem.setValues(indices[0].astype(dtype='int32'), indices[1].astype(dtype='int32'), D_values)
D_ekrem.setValues([1,2], [1,2], [1,2,3,4])

D_ekrem.assemble()
D_ekrem.getValues(2,2)

array(4.)

In [68]:
D11_block = multiplication[0:mesh.num_vertices(),0:mesh.num_vertices()]
D12_block = multiplication[0:mesh.num_vertices(),mesh.num_vertices():]
D21_block = multiplication[mesh.num_vertices():,0:mesh.num_vertices()]
D22_block = multiplication[mesh.num_vertices():,mesh.num_vertices():]

In [69]:
def local_to_global_block(local_matrix, location):
    global_matrix = np.zeros((mesh.num_vertices()*2,mesh.num_vertices()*2))
    if location == '11':
        global_matrix[0:mesh.num_vertices(),0:mesh.num_vertices()] = local_matrix
    if location == '12':
        global_matrix[0:mesh.num_vertices(),mesh.num_vertices():] = local_matrix
    if location == '21':
        global_matrix[mesh.num_vertices():,0:mesh.num_vertices()] = local_matrix
    if location == '22':
        global_matrix[mesh.num_vertices():,mesh.num_vertices():] = local_matrix
    return global_matrix

In [70]:
D11 = local_to_global_block(D11_block,'12')
D12 = local_to_global_block(D12_block,'12')
D21 = local_to_global_block(D21_block,'21')
D22 = local_to_global_block(D22_block,'22')

#D22[580][643]

In [71]:
#D_mat.getValues(580, 643)

In [72]:
# Build diagonal matrix v(802,802)
V = dolf.FunctionSpace(mesh, 'CG', 1)
v = dolf.interpolate(params.v, V)
v_array = np.repeat(v.vector()[:],2)
v_matrix = np.zeros((len(v_array), len(v_array)))
np.fill_diagonal(v_matrix, v_array)

In [73]:
v_mixed = dolf.interpolate(params.v, V)
v_mixed.vector()[:]

array([5.89458454e-195, 5.70417627e-178, 9.32939745e-162, 2.57890399e-146,
       1.20486302e-131, 9.51395504e-118, 1.26971262e-104, 2.86398474e-092,
       1.09183468e-080, 7.03499817e-070, 7.66111559e-060, 1.41007094e-050,
       4.38642624e-042, 2.30622470e-034, 2.04933676e-027, 3.07783945e-021,
       7.81267061e-016, 3.35176829e-011, 2.43035314e-007, 2.97841835e-004,
       6.16911599e-002, 2.15963866e+000, 1.27779202e+001, 1.27779202e+001,
       2.15963866e+000, 6.16911599e-002, 2.97841835e-004, 2.43035314e-007,
       3.35176829e-011, 7.81267061e-016, 3.07783945e-021])

In [74]:
inputArray = v_mixed.vector()[:]

newArray = np.zeros(2*len(inputArray),dtype=int)
newArray[::2] = inputArray
newArray

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  2,  0, 12,  0, 12,  0,  2,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [75]:
a = v_mixed.vector().vec()*v_mixed.vector().vec()
a[:]

array([0.00000000e+000, 0.00000000e+000, 8.89318163e-323, 6.65074578e-292,
       1.45169489e-262, 9.05153404e-235, 1.61217014e-208, 8.20240859e-184,
       1.19210296e-160, 4.94911993e-139, 5.86926921e-119, 1.98830006e-100,
       1.92407351e-083, 5.31867239e-068, 4.19978116e-054, 9.47309568e-042,
       6.10378221e-031, 1.12343507e-021, 5.90661638e-014, 8.87097586e-008,
       3.80579920e-003, 4.66403914e+000, 1.63275245e+002, 1.63275245e+002,
       4.66403914e+000, 3.80579920e-003, 8.87097586e-008, 5.90661638e-014,
       1.12343507e-021, 6.10378221e-031, 9.47309568e-042])